In [41]:
import numpy as np
from datetime import datetime
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import xarray as xr
import contextily as ctx
import shapely
import pint_xarray
from metloom.pointdata import SnotelPointData, USGSPointData

In [47]:
#pip install metloom

In [49]:
#pip install pint_xarray

In [4]:
# we will look at january through april 
# predict discharge based on the snotel data
# spike in discharge means rain on snow, we will have 2 classes, so binary classification
# final goal: get a decision tree to predict discharge spikes based on snotel data 
# truth rain from snotel, infer it from discharge


# SWE: snow water equivalent: determines how much water the snowpack contains

In [5]:
#pip install pint_xarray

In [6]:
# load in data 

snotel = xr.open_dataset('../../data/snotel/snotel_hourly_1980_2024.zarr')
usgs = xr.open_dataset('../../data/stream/usgs_instantaneous_1980_2024.zarr')

/opt/conda/lib/python3.11/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/opt/conda/lib/python3.11/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)


In [7]:
snotel

<xarray.Dataset> Size: 16MB
Dimensions:                    (date: 218948, site: 2)
Coordinates:
  * date                       (date) datetime64[ns] 2MB 1987-10-22T06:00:00 ...
  * site                       (site) object 16B '817:WA:SNTL' '990:WA:SNTL'
Data variables:
    ACCUMULATED PRECIPITATION  (date, site) float64 4MB ...
    SWE                        (date, site) float64 4MB ...
    AIR TEMP                   (date, site) float64 4MB ...
    SNOWDEPTH                  (date, site) float64 4MB ...
Attributes:
    Description:  Snotel Data
    Date Range:   1980-2024
    Sites:        Thunder Basin, Beaver Creek
    Frequency:    Hourly

In [8]:
usgs

<xarray.Dataset> Size: 27MB
Dimensions:    (date: 1134647, site: 2)
Coordinates:
  * date       (date) datetime64[ns] 9MB 1989-10-01T07:15:00 ... 2024-01-02T0...
  * site       (site) object 16B '12172000' '12175500'
Data variables:
    DISCHARGE  (date, site) float64 18MB ...
Attributes:
    Description:  Streamflow Data
    Date Range:   1980-2024
    Sites:        Thunder Creek, Beaver Creek
    Frequency:    15-Min

In [57]:
# we will just be working with daily data now 

from zoneinfo import ZoneInfo
# start data and end date
start_date = datetime(2014, 1, 1, tzinfo=ZoneInfo('UTC'))
end_date = datetime(2024, 1, 1, tzinfo=ZoneInfo('UTC'))

# USGS Sites
THUNDER_CREEK_USGS = ('12175500', 'Thunder Creek USGS Site')
BEAVER_CREEK_USGS = ('12172000', 'Beaver Creek USGS Site')

USGS_VARS = [USGSPointData.ALLOWED_VARIABLES.DISCHARGE,
            USGSPointData.ALLOWED_VARIABLES.STREAMFLOW,
            USGSPointData.ALLOWED_VARIABLES.SWE,
            USGSPointData.ALLOWED_VARIABLES.SNOWDEPTH,
            USGSPointData.ALLOWED_VARIABLES.SURFACETEMP]

# Create USGS DataFrame
thunder_basin_usgs = USGSPointData(*THUNDER_CREEK_USGS).get_hourly_data(start_date, end_date, variables=USGS_VARS)
beaver_creek_usgs = USGSPointData(*BEAVER_CREEK_USGS).get_hourly_data(start_date, end_date, variables=USGS_VARS)
usgs_df = pd.concat([thunder_basin_usgs, beaver_creek_usgs])

/opt/conda/lib/python3.11/site-packages/metloom/pointdata/usgs.py:216: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  if pd.to_datetime(sensor_df["datetime"])[0].tzinfo is None:
 Requested site, sensor(s), and date range resulted in no data returned.
 Requested site, sensor(s), and date range resulted in no data returned.
 Requested site, sensor(s), and date range resulted in no data returned.
 Requested site, sensor(s), and date range resulted in no data returned.
/opt/conda/lib/python3.11/site-packages/metloom/pointdata/usgs.py:216: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour an

In [67]:
usgs_df

,,geometry,DISCHARGE,DISCHARGE_units,datasource
datetime,site,,,,
2014-01-01 08:00:00+00:00,12175500,POINT Z (-121.0729 48.67263 1220),133.00,ft3/s,USGS
2014-01-01 09:00:00+00:00,12175500,POINT Z (-121.0729 48.67263 1220),133.00,ft3/s,USGS
2014-01-01 10:00:00+00:00,12175500,POINT Z (-121.0729 48.67263 1220),133.00,ft3/s,USGS
2014-01-01 11:00:00+00:00,12175500,POINT Z (-121.0729 48.67263 1220),133.00,ft3/s,USGS
2014-01-01 12:00:00+00:00,12175500,POINT Z (-121.0729 48.67263 1220),132.50,ft3/s,USGS
...,...,...,...,...,...
2024-01-02 03:00:00+00:00,12172000,POINT Z (-121.07343 48.77759 1610.23),225.00,ft3/s,USGS
2024-01-02 04:00:00+00:00,12172000,POINT Z (-121.07343 48.77759 1610.23),224.50,ft3/s,USGS
2024-01-02 05:00:00+00:00,12172000,POINT Z (-121.07343 48.77759 1610.23),223.00,ft3/s,USGS


In [58]:
# Create Snotel DataFrame

SNOTEL_VARS = [SnotelPointData.ALLOWED_VARIABLES.SNOWDEPTH,
            SnotelPointData.ALLOWED_VARIABLES.SWE,
            SnotelPointData.ALLOWED_VARIABLES.PRECIPITATION,
            SnotelPointData.ALLOWED_VARIABLES.PRECIPITATIONACCUM,
            SnotelPointData.ALLOWED_VARIABLES.TEMP,
            SnotelPointData.ALLOWED_VARIABLES.TEMPAVG,
            SnotelPointData.ALLOWED_VARIABLES.TEMPMAX,
            SnotelPointData.ALLOWED_VARIABLES.TEMPMIN,
            SnotelPointData.ALLOWED_VARIABLES.RH]

thunder_basin_data = SnotelPointData(*THUNDER_BASIN_SNOTEL).get_hourly_data(start_date, end_date, variables=SNOTEL_VARS)
beaver_creek_data = SnotelPointData(*BEAVER_CREEK_SNOTEL).get_hourly_data(start_date, end_date, variables=SNOTEL_VARS)
snotel_df = pd.concat([thunder_basin_data, beaver_creek_data])

No PRECIPITATION found for Thunder Basin Snotel Site
No AVG AIR TEMP found for Thunder Basin Snotel Site
No MAX AIR TEMP found for Thunder Basin Snotel Site
No MIN AIR TEMP found for Thunder Basin Snotel Site
No Relative Humidity found for Thunder Basin Snotel Site
No PRECIPITATION found for Beaver Creek Snotel Site
No AVG AIR TEMP found for Beaver Creek Snotel Site
No MAX AIR TEMP found for Beaver Creek Snotel Site
No MIN AIR TEMP found for Beaver Creek Snotel Site
No Relative Humidity found for Beaver Creek Snotel Site


In [ ]:
snotel_df

### Prediction

#### Predict discharge based on the snotel data

In [61]:
# the dataframe in case I need it, good for visualization 
# need to select 2014-01-01 to 2024-01-01

# for prediction, I only want ACCUMULATED PRECIPITATION, SWE, AIR TEMP, and SNOWDEPTH

snotel_var = snotel_df.filter(items=['ACCUMULATED PRECIPITATION', 'SWE', 'AIR TEMP', 'SNOWDEPTH'])

snotel_var

,,ACCUMULATED PRECIPITATION,SWE,AIR TEMP,SNOWDEPTH
datetime,site,,,,
2014-01-01 08:00:00+00:00,990:WA:SNTL,13.6,5.9,33.08,22.0
2014-01-01 09:00:00+00:00,990:WA:SNTL,13.6,5.9,32.72,22.0
2014-01-01 10:00:00+00:00,990:WA:SNTL,13.6,5.9,32.18,22.0
2014-01-01 11:00:00+00:00,990:WA:SNTL,13.6,5.9,32.00,22.0
2014-01-01 12:00:00+00:00,990:WA:SNTL,13.6,5.9,32.18,22.0
...,...,...,...,...,...
2024-01-01 04:00:00+00:00,817:WA:SNTL,29.4,9.1,25.88,21.0
2024-01-01 05:00:00+00:00,817:WA:SNTL,29.4,9.1,24.98,20.0
2024-01-01 06:00:00+00:00,817:WA:SNTL,29.4,9.1,24.98,21.0


In [64]:
# do some cleaning, we've got some null values 
# null values will be replaced with the mean of the row before and after it

def fill_null(df):
    df = df.copy() 
    df = df.reset_index()
    for col in df.columns:
        null_indices = df[df[col].isnull()].index
        for i in null_indices:
            before = df.loc[i-1, col] if i > 0 else np.nan
            after = df.loc[i+1, col] if i < len(df) - 1 else np.nan

            #Handle cases where either before or after is also NaN
            valid_values = [x for x in [before, after] if pd.notna(x)]
            if valid_values:
              df.loc[i, col] = np.mean(valid_values)
            #If both before and after are NaN, leave the NaN as is
            
    return df

In [77]:
snotel_cleaned = fill_null(snotel_var)

In [78]:
snotel_cleaned

,datetime,site,ACCUMULATED PRECIPITATION,SWE,AIR TEMP,SNOWDEPTH
0,2014-01-01 08:00:00+00:00,990:WA:SNTL,13.6,5.9,33.08,22.0
1,2014-01-01 09:00:00+00:00,990:WA:SNTL,13.6,5.9,32.72,22.0
2,2014-01-01 10:00:00+00:00,990:WA:SNTL,13.6,5.9,32.18,22.0
3,2014-01-01 11:00:00+00:00,990:WA:SNTL,13.6,5.9,32.00,22.0
4,2014-01-01 12:00:00+00:00,990:WA:SNTL,13.6,5.9,32.18,22.0
...,...,...,...,...,...,...
174845,2024-01-01 04:00:00+00:00,817:WA:SNTL,29.4,9.1,25.88,21.0
174846,2024-01-01 05:00:00+00:00,817:WA:SNTL,29.4,9.1,24.98,20.0
174847,2024-01-01 06:00:00+00:00,817:WA:SNTL,29.4,9.1,24.98,21.0
174848,2024-01-01 07:00:00+00:00,817:WA:SNTL,29.4,9.1,25.34,21.0


In [80]:
usgs_cleaned = fill_null(usgs_df)
usgs_cleaned

,datetime,site,geometry,DISCHARGE,DISCHARGE_units,datasource
0,2014-01-01 08:00:00+00:00,12175500,POINT Z (-121.0729 48.67263 1220),133.00,ft3/s,USGS
1,2014-01-01 09:00:00+00:00,12175500,POINT Z (-121.0729 48.67263 1220),133.00,ft3/s,USGS
2,2014-01-01 10:00:00+00:00,12175500,POINT Z (-121.0729 48.67263 1220),133.00,ft3/s,USGS
3,2014-01-01 11:00:00+00:00,12175500,POINT Z (-121.0729 48.67263 1220),133.00,ft3/s,USGS
4,2014-01-01 12:00:00+00:00,12175500,POINT Z (-121.0729 48.67263 1220),132.50,ft3/s,USGS
...,...,...,...,...,...,...
134702,2024-01-02 03:00:00+00:00,12172000,POINT Z (-121.07343 48.77759 1610.23),225.00,ft3/s,USGS
134703,2024-01-02 04:00:00+00:00,12172000,POINT Z (-121.07343 48.77759 1610.23),224.50,ft3/s,USGS
134704,2024-01-02 05:00:00+00:00,12172000,POINT Z (-121.07343 48.77759 1610.23),223.00,ft3/s,USGS
134705,2024-01-02 06:00:00+00:00,12172000,POINT Z (-121.07343 48.77759 1610.23),223.00,ft3/s,USGS


### Classification
#### Fit a model based on air temp, SWE, and snow_depth to predict discharge 

In [ ]:
# we have an issue, the SNOTEL and USGS dataframes are not the same length 
# i do not know why

In [69]:
# set grounds truths

# Snow or Rain on Snow: binary classification
# 0: Snow 
# 1: Rain on Snow 

discharge = usgs_df['DISCHARGE']
snotel_numeric = snotel_cleaned.filter(items=['ACCUMULATED PRECIPITATION', 'SWE', 'AIR TEMP', 'SNOWDEPTH'])

ground_truth = []

,ACCUMULATED PRECIPITATION,SWE,AIR TEMP,SNOWDEPTH
0,13.6,5.9,33.08,22.0
1,13.6,5.9,32.72,22.0
2,13.6,5.9,32.18,22.0
3,13.6,5.9,32.00,22.0
4,13.6,5.9,32.18,22.0
...,...,...,...,...
174845,29.4,9.1,25.88,21.0
174846,29.4,9.1,24.98,20.0
174847,29.4,9.1,24.98,21.0
174848,29.4,9.1,25.34,21.0


### We will start with a basic rolling mean classification....

In [91]:
def rolling_stats(data,var):
    # Calculate the rolling median and standard deviation
    rolling_median = data[var].rolling(window=24, center=True, min_periods=1).median()
    rolling_std = data[var].rolling(window=24, center=True, min_periods=1).std()

    return [rolling_median, rolling_std]

In [108]:
def assign_labels(data, var):
    mu = rolling_stats(data,var)[0]
    sigma = rolling_stats(data,var)[1]

    truths = []
    mask = data[var] > mu+sigma
    truths = mask

    for i in range(0,len(truths)):
        if truths[i] == False:
            truths[i]=0
        else:
            truths[i]=1

    return truths

In [109]:
ground_truths = assign_labels(usgs_cleaned, 'DISCHARGE')

/tmp/ipykernel_100/4084766458.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  truths[i]=0


In [110]:
ground_truths[0:50]

0     0
1     1
2     1
3     1
4     0
5     0
6     0
7     0
8     1
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
45    1
46    1
47    1
48    0
49    0
Name: DISCHARGE, dtype: object

## Decision Tree Regression 
#### Predict float discharge values

In [ ]:
from sklearn import tree

X = # snotel data 
y = # discharge values 
clf = tree.DecisionTreeRegressor()
clf = clf.fit(X, y)



## Decision Tree Classification

#### Predicts integer values 0 (snow)or 1 (rain on snow)

In [ ]:
# will use sklearn's decision tree for now....

from sklearn import tree
clf = tree.DecisionTreeClassifier()